In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("ember_yearly_electricity-generation_share.csv")

In [3]:
df.head()

,entity,entity_code,is_aggregate_entity,date,series,is_aggregate_series,generation_twh,generation_share_pct,generation_kwh_per_capita,generation_yoy_change_twh,generation_yoy_change_pct,generation_share_yoy_change_pct_points,generation_yoy_change_kwh_per_capita
0,Germany,DEU,False,2022,Bioenergy,False,48.60,8.57,577.98,-0.50,-1.02,0.09,-5.95
1,Germany,DEU,False,2022,Coal,False,179.93,31.74,2139.83,15.28,9.28,3.29,181.72
2,Germany,DEU,False,2022,Gas,False,79.05,13.94,940.11,-11.26,-12.47,-1.66,-133.91
3,Germany,DEU,False,2022,Hydro,False,17.62,3.11,209.55,-2.04,-10.38,-0.29,-24.26
4,Germany,DEU,False,2022,Nuclear,False,34.71,6.12,412.79,-34.42,-49.79,-5.82,-409.34


In [14]:
import plotly.graph_objects as go

def plot_electricity_gen_share(df, country_name, save_as_html=False, save_as_png=True):

    df_cntry = df[(df["entity"] == country_name) & (df["series"] != "Total generation")]
    df_totals = df[(df["entity"] == country_name) & (df["series"] == "Total generation")]

    fig = go.Figure()

    for source in df_cntry["series"].unique():
        subset = df_cntry[df_cntry["series"] == source]
        fig.add_trace(go.Bar(
            x=subset["date"],
            y=subset["generation_share_pct"],
            name=source,
            text=subset["generation_share_pct"].round(1).astype(str) + "%",
            textposition="inside",
            insidetextanchor="middle"
        ))

    for _, row in df_totals.iterrows():
        year = row["date"]
        total = row["generation_twh"]

        total_height = df_cntry[df_cntry["date"] == year]["generation_share_pct"].sum()

        fig.add_annotation(
            x=year,
            y=total_height + 3,
            text=f"Total: {total:.1f} TWh",
            showarrow=False,
            font=dict(size=14, color="black"),
            align="center"
        )

    fig.update_layout(
        height=800,
        width=1200,
        xaxis=dict(
            title="Year",
            tickmode="array",
            tickvals=df_cntry["date"].unique(),
            ticktext=[str(year) for year in df_cntry["date"].unique()]
        ),
        title=dict(
            text=f"<b>{country_name} Electricity Generation Share by Source (2022–2024)</b>",
            font=dict(size=20),
        ),
        yaxis_title="Generation Share (%)",
        barmode="stack",
        template="plotly_white",
        legend_title="Source",
    )

    if save_as_html:
        fig.write_html(f"{country_name}_electricity_gen_share.html")
    
    if save_as_png:
        fig.write_image(f"{country_name}_electricity_gen_share.png", format="png", height=800, width=1200)
    fig.show()

## Plot 1 - Bar plot of electricity generation share by source for germany from 2022 to 2024 

In [15]:
plot_electricity_gen_share(df, "Germany", save_as_html=True)

## Plot 2 - Bar plot of electricity generation share by source for denmark from 2022 to 2024

In [16]:
plot_electricity_gen_share(df, "Denmark", save_as_html=True)

## Plot 3 - Bar plot of electricity generation share by source for france from 2022 to 2024

In [17]:
plot_electricity_gen_share(df, "France", save_as_html=True)

## Plot 4 - Bar plot of electricity generation share by source for the world from 2022 to 2024

In [18]:
plot_electricity_gen_share(df, "World", save_as_html=True)

In [13]:
df[(df["entity"].isin(["Germany", "Denmark", "France", "World"])) & (df["series"] == "Total generation")][["entity", "date", "series", "generation_twh"]].to_clipboard()

# Plot 5 - Bar plot of every country's electricity generation share by source for every year from 2022 to 2024

In [31]:
from plotly.subplots import make_subplots
from plotly.express import colors
import plotly.graph_objects as go

df_ger = df[(df["entity"] == "Germany") & (df["series"] != "Total generation")]
df_den = df[(df["entity"] == "Denmark") & (df["series"] != "Total generation")]
df_fra = df[(df["entity"] == "France") & (df["series"] != "Total generation")]
df_wrld = df[(df["entity"] == "World") & (df["series"] != "Total generation")]

df_ger_totals = df[(df["entity"] == "Germany") & (df["series"] == "Total generation")]
df_den_totals = df[(df["entity"] == "Denmark") & (df["series"] == "Total generation")]
df_fra_totals = df[(df["entity"] == "France") & (df["series"] == "Total generation")]
df_wrld_totals = df[(df["entity"] == "World") & (df["series"] == "Total generation")]

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(f"<b>Germany</b>", f"<b>Denmark</b>", f"<b>France</b>", f"<b>World</b>"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.125
)

datasets = [
    (df_ger, df_ger_totals, "Germany", 1, 1),
    (df_den, df_den_totals, "Denmark", 1, 2),
    (df_fra, df_fra_totals, "France", 2, 1),
    (df_wrld, df_wrld_totals, "World", 2, 2),
]

color_map = {
    "Coal": colors.qualitative.Plotly[0],
    "Gas": colors.qualitative.Plotly[1],
    "Hydro": colors.qualitative.Plotly[2],
    "Nuclear": colors.qualitative.Plotly[3],
    "Solar": colors.qualitative.Plotly[4],
    "Wind": colors.qualitative.Plotly[5],
    "Bioenergy": colors.qualitative.Plotly[6],
    "Other fossil": colors.qualitative.Plotly[7],
    "Other renewables": colors.qualitative.Plotly[8],
}

for df_country, df_totals, country_name, row, col in datasets:
    for source in df_country["series"].unique():
        subset = df_country[df_country["series"] == source]
        fig.add_trace(
            go.Bar(
                x=subset["date"],
                y=subset["generation_share_pct"],
                name=source,
                text=subset["generation_share_pct"].round(1).astype(str) + "%",
                textposition="inside",
                insidetextanchor="middle",
                legendgroup=source,
                showlegend=(row == 1 and col == 1), 
                marker=dict(color=color_map.get(source, "#CCCCCC"))
            ),
            row=row, col=col
        )

    for _, row_total in df_totals.iterrows():
        year = row_total["date"]
        total = row_total["generation_twh"]

        total_height = df_country[df_country["date"] == year]["generation_share_pct"].sum()

        fig.add_annotation(
            x=year,
            y=total_height + 3,  
            text=f"{total:.1f} TWh",
            showarrow=False,
            font=dict(size=13, color="black"),
            align="center",
            row=row, col=col
        )

fig.update_layout(
    height=1300,
    autosize=True,
    title=dict(
        text="<b>Electricity Generation Share by Source (2022–2024)</b>",
        font=dict(size=22),
        x=0.5
    ),
    barmode="stack",
    template="plotly_white",
    legend_title="Source",
)

for r in range(1, 3):
    for c in range(1, 3):
        fig.update_xaxes(
            tickmode="array",
            tickvals=["2022", "2023", "2024"],
            ticktext=["2022", "2023", "2024"],
            row=r, col=c
        )

fig.update_xaxes(title_text="Year", row=2, col=1, title_standoff=10)
fig.update_xaxes(title_text="Year", row=2, col=2, title_standoff=10)
fig.update_yaxes(title_text="Generation Share (%)", row=1, col=1)
fig.update_yaxes(title_text="Generation Share (%)", row=2, col=1)

fig.show()
#fig.write_html("all_entities_electricity_gen_share.html", include_plotlyjs="cdn", full_html=True)
fig.write_image("all_entities_electricity_gen_share.png", format="png", height=1300, width=2000)

# Plot 7 Per Capita

In [20]:
import pandas as pd 

df_1 = pd.read_csv("ember_yearly_electricity-generation_per_capita.csv")

In [21]:
df_1

,entity,entity_code,is_aggregate_entity,date,series,is_aggregate_series,generation_twh,generation_share_pct,generation_kwh_per_capita,generation_yoy_change_twh,generation_yoy_change_pct,generation_share_yoy_change_pct_points,generation_yoy_change_kwh_per_capita
0,Germany,DEU,False,2022,Bioenergy,False,48.60,8.57,577.98,-0.50,-1.02,0.09,-5.95
1,Germany,DEU,False,2022,Coal,False,179.93,31.74,2139.83,15.28,9.28,3.29,181.72
2,Germany,DEU,False,2022,Gas,False,79.05,13.94,940.11,-11.26,-12.47,-1.66,-133.91
3,Germany,DEU,False,2022,Hydro,False,17.62,3.11,209.55,-2.04,-10.38,-0.29,-24.26
4,Germany,DEU,False,2022,Nuclear,False,34.71,6.12,412.79,-34.42,-49.79,-5.82,-409.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,World,NaN,True,2024,Other fossil,False,874.71,2.83,107.19,0.56,0.06,-0.12,0.07
116,World,NaN,True,2024,Other renewables,False,88.67,0.29,10.87,-0.25,-0.28,-0.01,-0.03
117,World,NaN,True,2024,Solar,False,2128.58,6.89,260.83,468.83,28.25,1.29,57.45
118,World,NaN,True,2024,Total generation,True,30913.94,100.00,3788.17,1250.91,4.22,0.00,153.29


In [32]:
from plotly.subplots import make_subplots
from plotly.express import colors
import plotly.graph_objects as go

df_1_ger = df_1[(df_1["entity"] == "Germany") & (df_1["series"] != "Total generation")]
df_1_den = df_1[(df_1["entity"] == "Denmark") & (df_1["series"] != "Total generation")]
df_1_fra = df_1[(df_1["entity"] == "France") & (df_1["series"] != "Total generation")]
df_1_wrld = df_1[(df_1["entity"] == "World") & (df_1["series"] != "Total generation")]

df_1_ger_totals = df_1[(df_1["entity"] == "Germany") & (df_1["series"] == "Total generation")]
df_1_den_totals = df_1[(df_1["entity"] == "Denmark") & (df_1["series"] == "Total generation")]
df_1_fra_totals = df_1[(df_1["entity"] == "France") & (df_1["series"] == "Total generation")]
df_1_wrld_totals = df_1[(df_1["entity"] == "World") & (df_1["series"] == "Total generation")]

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(f"<b>Germany</b>", f"<b>Denmark</b>", f"<b>France</b>", f"<b>World</b>"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.125
)

datasets = [
    (df_1_ger, df_1_ger_totals, "Germany", 1, 1),
    (df_1_den, df_1_den_totals, "Denmark", 1, 2),
    (df_1_fra, df_1_fra_totals, "France", 2, 1),
    (df_1_wrld, df_1_wrld_totals, "World", 2, 2),
]

color_map = {
    "Coal": colors.qualitative.Plotly[0],
    "Gas": colors.qualitative.Plotly[1],
    "Hydro": colors.qualitative.Plotly[2],
    "Nuclear": colors.qualitative.Plotly[3],
    "Solar": colors.qualitative.Plotly[4],
    "Wind": colors.qualitative.Plotly[5],
    "Bioenergy": colors.qualitative.Plotly[6],
    "Other fossil": colors.qualitative.Plotly[7],
    "Other renewables": colors.qualitative.Plotly[8],
}

for df_1_country, df_1_totals, country_name, row, col in datasets:
    for source in df_1_country["series"].unique():
        subset = df_1_country[df_1_country["series"] == source]
        fig.add_trace(
            go.Bar(
                x=subset["date"],
                y=subset["generation_kwh_per_capita"],
                name=source,
                text=subset["generation_share_pct"].round(1).astype(str) + "%",
                textposition="inside",
                insidetextanchor="middle",
                legendgroup=source,
                showlegend=(row == 1 and col == 1), 
                marker=dict(color=color_map.get(source, "#CCCCCC"))
            ),
            row=row, col=col
        )

    for _, row_total in df_1_totals.iterrows():
        year = row_total["date"]
        total = row_total["generation_kwh_per_capita"]

        total_height = df_1_country[df_1_country["date"] == year]["generation_kwh_per_capita"].sum()

        fig.add_annotation(
            x=year,
            y=total_height + 7,  
            text=f"{total:.1f} TWh",
            showarrow=False,
            font=dict(size=13, color="black"),
            align="center",
            row=row, col=col
        )

fig.update_layout(
    height=1300,
    autosize=True,
    title=dict(
        text="<b>Electricity Generation per Capita by Source (2022–2024)</b>",
        font=dict(size=22),
        x=0.5
    ),
    barmode="stack",
    template="plotly_white",
    legend_title="Source",
)

for r in range(1, 3):
    for c in range(1, 3):
        fig.update_xaxes(
            tickmode="array",
            tickvals=["2022", "2023", "2024"],
            ticktext=["2022", "2023", "2024"],
            row=r, col=c
        )

fig.update_xaxes(title_text="Year", row=2, col=1, title_standoff=10)
fig.update_xaxes(title_text="Year", row=2, col=2, title_standoff=10)
fig.update_yaxes(title_text="Generation per Capita (%)", row=1, col=1)
fig.update_yaxes(title_text="Generation per Capita (%)", row=2, col=1)

fig.show()
fig.write_html("all_entities_electricity_gen_capita.html", include_plotlyjs="cdn", full_html=True)
fig.write_image("all_entities_electricity_gen_capita.png", format="png", height=1300, width=2000)